# 🔍 Evaluation

This notebook uses perceptual similarity metrics to evaluate how closely our model was able to reproduce the affect of Cinestill 800T

### Setup

---

Let's install some necessary dependencies and set global variables.

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
# Autoroot
import autorootcwd

In [ ]:
import cv2 as cv

import os

# Imports
import torch
from matplotlib import pyplot as plt

# Local modules
from src.data.components.paired import PairedDataset
from src.utils.utils import undo_transforms

In [ ]:
# pieapp - batching works
from src.eval import PieAPP

app = PieAPP()
x, y = torch.rand(2, 3, 256, 256), torch.rand(2, 3, 256, 256) # fake data
app.update(x, y)
print(app.compute())
print()

In [ ]:
# pieapp - batching works
app = PieAPP()
app.update(x[0].unsqueeze(0), y[0].unsqueeze(0))
app.update(x[1].unsqueeze(0), y[1].unsqueeze(0))
print(app.compute())

In [ ]:
# sim-dist works
from src.eval import PieAPP

app = PieAPP()
app.update(x, x)
app.compute()

In [ ]:
from src.eval import FID

fid = FID()
x, y = torch.rand(2, 3, 256, 256), torch.rand(2, 3, 256, 256) # fake data
fid.update(x, y)
fid.compute()

### Load the `PairedDataset`

---

In [ ]:
# Constants
RAW_DIR = os.getcwd()
DATA_DIR = os.path.join(RAW_DIR, 'data')
# Instantiate paths
film_paired_dir = os.path.join(DATA_DIR, "paired", "processed", "film")
digital_paired_dir = os.path.join(DATA_DIR, "paired", "processed", "digital")

In [ ]:
# Instantiate dataset
digital_film_data = PairedDataset(
    image_dirs=(film_paired_dir, digital_paired_dir),
)

print(f"✅ Loaded {len(digital_film_data)} paired samples")

In [ ]:
# Inspect samples
film, digital = digital_film_data[0]
print(f"Film image: {film.shape}, Digital image {digital.shape}")

print(type(film), type(digital))

# Show sample
fig, axs = plt.subplots(ncols=2, figsize=(8, 4))
axs[0].imshow(undo_transforms(film).numpy())
axs[1].imshow(undo_transforms(digital).numpy())

#### Make sure we have Tensors to give to the Eval Metric

In [ ]:
print(type(film), type(digital))
print(type(film.unsqueeze(0)), type(digital.unsqueeze(0)))

### Try using our Evaluation Metrics
---

In [ ]:
from src.eval.pie_app import PieAPPMetric

x = film.unsqueeze(0)
y = digital.unsqueeze(0)

metric = PieAPPMetric()

metric.update(x, x)
loss: torch.Tensor = metric.compute()
print(f"PieAPP loss: {loss:0.4f}")

In [ ]:
# Define metrics
from torchmetrics import MetricCollection
from src.eval import SSIM, PSNR, LPIPS, PieAPP


metrics = MetricCollection(
    {
        "ssim": SSIM(),
        "psnr": PSNR(),
        "lpips": LPIPS(),
        "pieapp": PieAPP(),
    }
)